# Thinking in Coroutines

![title](/coroutines_example.png)

In [ ]:
import asyncio

if __name__ == '__main__':
    loop = asyncio.get_event_loop()
    loop.runforever()

In [2]:
import asyncio
import datetime

def anything(i):
    print(i, datetime.datetime.now())
    
if __name__ == '__main__':
    loop = asyncio.get_event_loop()
    loop.call_later(2, loop.stop)
    for i in range(1, 4):
        loop.call_soon(anything, i)
    try:
        loop.run_forever()
    finally:
        loop.close()

RuntimeError: Cannot close a running event loop

1 2018-05-31 11:59:06.971381
2 2018-05-31 11:59:06.971503
3 2018-05-31 11:59:06.971703
